## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/aft


## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_gbdt'


# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'aft'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6295186698940518
Integrated Brier Score: 0.16813993667371294
{'model': 'aft', 'dataset': 'METABRIC', 'cindex_train': [0.6702316377546303], 'cindex_test': [0.6295186698940518], 'ibs_train': [0.15927975303092673], 'ibs_test': [0.16813993667371294]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6217821392751086
Integrated Brier Score: 0.17752026325939715
{'model': 'aft', 'dataset': 'METABRIC', 'cindex_train': [0.6925262431959187], 'cindex_test': [0.6217821392751086], 'ibs_train': [0.15972973757444572], 'ibs_test': [0.17752026325939715]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6177071149614416
Integrated Brier Score: 0.1785095559836587
{'model': 'aft', 'dataset': 'MET

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,aft,METABRIC,0.670232,0.629519,0.159280,0.168140
1,aft,METABRIC,0.692526,0.621782,0.159730,0.177520
2,aft,METABRIC,0.684610,0.617707,0.159751,0.178510
3,aft,METABRIC,0.692944,0.684020,0.156818,0.154720
4,aft,METABRIC,0.715716,0.649539,0.146987,0.170834
5,aft,FLCHAIN,0.807131,0.806345,0.091055,0.094490
6,aft,FLCHAIN,0.807045,0.807871,0.091236,0.093323
7,aft,FLCHAIN,0.809408,0.784739,0.091405,0.097013
8,aft,FLCHAIN,0.808131,0.784731,0.091888,0.097393
9,aft,FLCHAIN,0.804331,0.801448,0.092932,0.091312


## TCGA Train, Test, Evaluation

In [10]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'aft'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5640883977900553
Integrated Brier Score: 0.22889628706169132
{'model': 'aft', 'dataset': 'BLCA', 'cindex_train': [0.8111287289354328], 'cindex_test': [0.5640883977900553], 'ibs_train': [0.2099599067769279], 'ibs_test': [0.22889628706169132]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5368651010376844
Integrated Brier Score: 0.22530331194015538
{'model': 'aft', 'dataset': 'BLCA', 'cindex_train': [0.8697458727508811], 'cindex_test': [0.5368651010376844], 'ibs_train': [0.18894044938526372], 'ibs_test': [0.22530331194015538]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.23596176821983272
Integrated Brier Score: 0.22393898755898825
{'model': 'aft', 'dataset': 'BLCA', 'cindex

KeyboardInterrupt: 

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,aft,BLCA,0.811129,0.564088,0.209960,0.228896
1,aft,BLCA,0.869746,0.536865,0.188940,0.225303
2,aft,BLCA,0.613669,0.235962,0.211702,0.223939
3,aft,BLCA,0.731978,0.549828,0.210370,0.225966
4,aft,BLCA,0.754069,0.491273,0.213425,0.216176
5,aft,BRCA,0.797414,0.649740,0.157666,0.204126
6,aft,BRCA,0.867366,0.558092,0.155974,0.194790
7,aft,BRCA,0.875118,0.461940,0.118753,0.188972
8,aft,BRCA,0.686560,0.522830,0.194727,0.186000
9,aft,BRCA,0.869480,0.686987,0.140081,0.206410
